# Random Forest y la clasificación clientes
Décimo primer día del reto. Ayer predecíamos cosas como precios o tiempos Hoy vamos a hacer algo diferente. Vamos a predecir si un cliente va a responder SÍ o NO a una campaña de marketing. Volveremos a usar _Random Forest_, pero ahora en modo clasificación. _Random Forest_ puede predecir números (a eso llamamos regresión) pero también puede hacer clasificaciones. Entonces, en vez de adivinar números, asignamos categorías. En este caso tenemos frente a nosotros un modelo de clasificación binaria, pues sólo hay dos categorías. El modelo elegido se llama `RandomForestClassifier`. _Let's go!_

## Simulación de datos: ¿Quién respondería a la campaña?
Partimos de un _DataFrame_ generado artificialmente. Éstas serán las características:

- Edad.

- Ingresos.

- Antigüedad en la compañía.

- Compras previas.

- ¿Respondió? (nuestro objetivo: 1 = Sí, 0 = No).

In [1]:
# Librerías necesarias y creación dell DataFrame con los datos.
import numpy as np
import pandas as pd

np.random.seed(25)
n = 1000
df = pd.DataFrame({
    'edad': np.random.randint(18, 70, n),
    'ingresos': np.random.normal(30000, 10000, n).astype(int),
    'antiguedad': np.random.randint(1, 10, n),
    'compras_previas': np.random.poisson(5, n)
})
# Simulación de la respuesta con base en otros datos del DataFrame.
df['respondio'] = ((df['compras_previas'] > 6) | (df['ingresos'] > 40000)) & (df['edad'] < 50)
df['respondio'] = df['respondio'].astype(int)
df.head()

,edad,ingresos,antiguedad,compras_previas,respondio
0,22,19244,9,2,0
1,44,16499,1,2,0
2,33,35318,6,6,0
3,41,20743,5,3,0
4,62,29606,6,6,0


## División y escalado
Ya tenemos los datos. Ahora vamos a separar la variable objetivo (respondio) del resto de las características. Luego dividiremos los datos en un conjunto para entrenar el modelo (`train`) y otro para probarlo (`test`), reservando el 20% para prueba, que es una convención en este tipo de pruebas. Finalmente, escalaremos las variables numéricas, es decir, las ajustamos para que todas estén en la misma escala y facilitar que el modelo aprenda correctamente sin dejarse influir por diferencias de unidades o rangos (esto de logra con  `StandardScaler`).

In [2]:
# Es escalador y divisor.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Nos deshacemos de la variablle objetivo.
X = df.drop('respondio', axis=1)

# Mantenemos la variable ibjetivo aislada.
y = df['respondio']

# Creación de las variables de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Nuestro escalador en acción. Variables escaladas.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Antes de continuar, quiero que comprendamos mejor cómo funciona este modelo.

## ¿Qué es el modelo _Random Forest Classifier_?
Recuerdo una tarea de clasificación. Fue en el proyecto del tercer día, sobre los sobrevivientes del Titanic. Para ese proyecto usamos una regresión logística porque los datos mostraban una relación lineal y más simple. Para obtener predicciones precisas, a veces con modelos simples como `LogisticRegression` bastan. Otras veces no. El conjunto de datos de nuestro _DataFrame_ actual necesita de un modelo que se desempeñe bien cuando hay relaciones complejas y no lineales entre variables. _Random Forest Classifier_ es un modelo de clasificación más robusto que `LogisticRegression`.

## Entrenamiento del modelo y predicción
Con la información clara y las variables de entrenamiento y prueba divididas, vamos a configurar el número de árboles de nuestro bosque. Serán 100. Entrenaremos al modelo y almacenaremos las predicciónes en una variable.

In [3]:
# Aquí está el modelo.
from sklearn.ensemble import RandomForestClassifier

# Los 100 árboles de los que hablaba.
model = RandomForestClassifier(random_state=0, n_estimators=100)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

## Evaluación
Vamos a ver qué tan bien predice el modelo, interpretando su precisión y la matriz de confusión, metricas esenciales en este tipo de proyectos. Si acierta mucho, estamos del otro lado.

In [4]:
# Vamos a importar algunas métricas. 
from sklearn.metrics import accuracy_score, confusion_matrix

print("Precisión:", accuracy_score(y_test, y_pred))
print()
print("Matriz de confusión:")
print()
print(confusion_matrix(y_test, y_pred))

Precisión: 0.99

Matriz de confusión:

[[153   0]
 [  2  45]]


## Conclusión
- La precisión fue alta. Eso indica una buena capacidad para distinguir clientes que responden o ignoran la campaña. La precisión máxima es 1, así que 0.99 es rozar la perfección.

- La matriz de confusión indica cuántos ‘sí’ y ‘no’ acertó o confundió el modelo. En este caso de 153 negativos (con respuesta 'no') el modelo acertó a todos. De 47 positivos (respuesta 'sí') el modelo predijo correctamente 45, es decir falló en dos ocasiones.

## Conclusión
Cuando el futuro es incierto y no basta una simple línea, el RandomForestClassifier te ayuda a elegir—no sólo a predecir. Así, identificar los sí y los no de tu campaña resulta mucho más confiable y robusto. La magia de la clasificación, con potencia y estilo.

¿Necesitas adaptación a otro dominio (salud, deporte, crédito)? ¿Deseas añadir interpretación visual del bosque o análisis de importancia de variables? ¡Sólo dilo y afinamos el notebook!